<a href="https://colab.research.google.com/github/codesongs/codestates_TP2/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 모델 훈련
개별 파일(MF.py, KNN.py)에서도 훈련이 가능하지만, 최상위 모듈인 train.py를 사용하여 학습을 진행할 수 있습니다.</br>
train.py 모듈을 터미널에서 실행하는 것으로 학습할 수 있습니다.</br>
train.py와 함께 사용할 수 있는 arguments들은 다음과 같습니다.</br>

**mf 모델 학습용 arguments**
- --mf : Matrix Factorization 모델의 학습 여부를 결정하는 인자입니다. --mf 를 입력하는 경우 학습을 진행하며, --no-mf를 입력하는 경우 해당 모델의 학습을 진행하지 않습니다.(입력하지 않았을 경우에는 학습 진행을 하도록 되어 있습니다.)
- -k : Matrix Factorization 행렬의 크기를 결정하는 인자값입니다. (default = 200)
- -e 또는 --epochs : mf 모델의 학습 횟수를 결정하는 인자입니다. (default = 1)
- -b 또는 --batch_size : 학습 시 사용하는 batch_size를 결정하는 인자입니다.(default = 512)

**knn 모델 학습용 arguments**
- --knn : 최근접 이웃을 활용한 콘텐츠 기반 필터링 모델의 학습 여부를 결정하는 인자입니다. --knn을 입력하는 경우 학습을 진행하며, word2vec 모델과 cbf_data, knn모델을 models 폴더 경로에 생성합니다.(이미 존재하는 경우에는 업데이트) --no-knn을 입력하면 해당 모델의 학습을 진행하지 않습니다.(입력하지 않았을 경우에는 학습 진행을 하도록 되어 있습니다.)
- --vector_size : word2vec 학습에 생성하는 행렬 차원의 수를 결정하는 인자입니다.(default = 100)
- --pretrained : 영화 장르 임베딩을 적용할 때 사용하는 gensim의 사전학습 모델을 결정하는 인자입니다. (default = 'glove-twitter-100')

In [ ]:
#CLI
#mf모델만 3 epochs로 학습하라는 명령어(knn 모델은 학습하지 않음)
#두가지 모델을 전무 학습시키고 싶다면 !python trian.py 명령어를 사용하시면 됩니다.
!python train.py --no-knn -e 3

2023-06-28 07:53:59.530928: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 07:53:59.574179: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-28 07:53:59.574713: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 07:54:00.381440: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Epoch 1/3
1251/1251 [==============================] - 25s 19ms/step - loss: 5.7587 - val_loss: 0.9398
Epoch 2/3
1251/1251 [==============================] - 23s 18ms/step - loss: 0.8581 - val_loss: 0.8353
Epoch 3/3
1251/1251 [==============================] - 25s 20ms/step - loss: 0.7570 - 

### 인기도 기반 추천 시스템 테스트
베이스라인 모델의 인기도 기반 추천은 사용자들이 가장 많이 평점을 남긴 영화 순으로 추천하도록 설계되어 있습니다.</br>
관련 코드는 models 폴더의 impersonal.py 코드에 작성되어 있습니다.

In [ ]:
from models.impersonal import popular
popular(10)

[2858, 260, 1196, 1210, 480, 2028, 589, 2571, 1270, 593]

### 협업 필터링 모델 테스트
Matrix Factorization 기반 모델 코드의 경우 train.py를 통해 학습한 mf.h5 파일을 로드하여 예측을 진행합니다.

In [ ]:
from models.MF import MF
mf = MF('./models/mf.h5')

In [ ]:
import numpy as np
from sklearn.metrics import ndcg_score

In [ ]:
from utils.Dataloader import load_ratings
ratings_df = load_ratings('datasets')

In [ ]:
user_ids = ratings_df['userId'].unique()  # Get the first 10 unique user IDs

predicted_scores = []  # Collect predicted scores for selected users
true_ratings_dict = {}

for user_id in user_ids:
    recommended_movies = mf.predict(int(user_id))[0]
    predicted_scores.append(recommended_movies)
    true_ratings_user_df = ratings_df[ratings_df['userId'] == user_id]
    true_ratings_user_df = true_ratings_user_df.sort_values(by='rating', ascending=False)
    true_movie_ids = true_ratings_user_df['movieId'].values.tolist()[:20]
    true_ratings_dict[user_id] = true_movie_ids

In [ ]:
recall = []
for user_id in user_ids:
    asdf =calculate_recall(true_ratings_dict[user_id], predicted_scores[user_id-1])
    recall.append(asdf)

In [ ]:
def calculate_recall(true_items, predicted_items, k=20):
    true_positive = len(set(true_items) & set(predicted_items))  # 실제 양성과 예측된 양성의 교집합

    recall_at_k = true_positive / k
    return recall_at_k


In [ ]:
recall20 = np.mean(recall)
recall20

0.09245033112582782

In [ ]:
user_ids = ratings_df['userId'].unique()  # Get the first 10 unique user IDs

predicted_scores = []  # Collect predicted scores for selected users
true_ratings_dict = {}

for user_id in user_ids:
    recommended_movies = mf.predict(int(user_id))
    predicted_scores.append(recommended_movies)
    true_ratings_user_df = ratings_df[ratings_df['userId'] == user_id]
    true_ratings_user_df = true_ratings_user_df.sort_values(by='rating', ascending=False)
    true_movie_ids = true_ratings_user_df['movieId'].values.tolist()[:20]
    true_ratings_dict[user_id] = true_movie_ids



In [ ]:
MF.train()

AttributeError: type object 'MF' has no attribute 'train'

In [ ]:
ndcg20 = []

for user_id in user_ids:
    true_ratings_info = true_ratings_dict[user_id]
    predicted_scores_user = predicted_scores[user_id - 1][0]
    common_items = set(true_ratings_info) & set(predicted_scores_user)
    indices = [predicted_scores_user.index(item) for item in common_items]
    exclude_indices = indices
    predicted_scores_user2 = predicted_scores[user_id - 1][1]

    for idx in range(len(predicted_scores_user2)):
        if idx not in exclude_indices:
            predicted_scores_user2[idx] = 0
        else:
            predicted_scores_user2[idx] = 1
    ndcg_value = ndcg_at_k(predicted_scores_user2, 20)
    ndcg20.append(ndcg_value)

average_ndcg20 = sum(ndcg20) / len(user_ids)

In [ ]:
print(f"NDCG@20: {average_ndcg20:.4f}")
print(f"리콜@20: {recall20:.4f}")

NDCG@20: 0.0964
리콜@20: 0.0925


In [ ]:
import numpy as np

def ndcg_at_k(ranked_list, k):
    idcg = np.sum(1 / np.log2(np.arange(2, k + 2)))
    dcg = np.sum(ranked_list[:k] / np.log2(np.arange(2, k + 2)))
    ndcg = dcg / idcg
    return ndcg


### 콘텐츠 기반 필터링 모델 테스트
최근접 이웃 알고리즘을 활용한 콘텐츠 기반 필터링 모델의 경우 train.py를 통해 학습한 knn.joblib 파일을 로드하여 예측을 진행합니다.

In [ ]:
from models.KNN import KNN

knn = KNN('models/knn.joblib')

In [ ]:
USER_ID = 1
TOP_NUM=20
knn.predict(USER_ID, TOP_NUM)

array([1205, 2081, 2080,  258, 1566,  673, 2054, 1215,  558,   70, 1196,
       1264,  610, 1210, 2322,  919,  107, 2045, 1473, 3266], dtype=int64)